In [1]:
import requests
import json
import time
import requests.packages.urllib3 as urllib3
 
urllib3.disable_warnings()
# Need to install requests package for python
# pip install requests
 
user = "gdluserTest"
pwd = "123456"
host_url = "http://192.168.28.79"
headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
'''
Get token for netbrain
'''
TENANT = 'Initial Tenant'
DOMAIN = 'GE Test'


In [2]:
def getTokens():
    login_api_url = r"/ServicesAPI/API/V1/Session"
    Login_url = host_url + login_api_url
    data = {
        "username": user,
        "password": pwd
    }
    token = requests.post(Login_url, data=json.dumps(
        data), headers=headers, verify=False)
    if token.status_code == 200:
        print(token.json())
        return token.json()["token"]
    else:
        return "error"
# get token
token = getTokens()
headers["Token"] = token

{'token': '712f1d5e-d663-4481-9672-e5d40b2ad58f', 'statusCode': 790200, 'statusDescription': 'Success.'}


In [3]:
def get_tenant_domain_id():
    tenant_id_url = '/ServicesAPI/API/V1/CMDB/Tenants'
    full_url = host_url + tenant_id_url
    data = requests.get(full_url,headers=headers,verify=False)
    tenant_id = ""
    domain_id = ""
    print(data.json())
    if data.status_code == 200:
        for tenant in data.json()['tenants']:
            if TENANT == tenant['tenantName']:
                tenant_id = tenant['tenantId']
        if tenant_id:
            domain_id_url = '/ServicesAPI/API/V1/CMDB/Domains'
            full_domain_url = host_url +domain_id_url
            domain_data = requests.get(full_domain_url,params={'tenantId':tenant_id},headers=headers,verify=False)
            print(domain_data.json())
            if domain_data.status_code == 200:
                for domain in domain_data.json()['domains']:
                    if DOMAIN == domain['domainName']:
                        domain_id = domain['domainId']
        return tenant_id,domain_id
    else:
        return tenant_id,domain_id
    
tenant_id,domain_id = get_tenant_domain_id()
print(tenant_id,domain_id)

{'tenants': [{'tenantId': 'fb24f3f0-81a7-1929-4b8f-99106c23fa5b', 'tenantName': 'Initial Tenant'}, {'tenantId': 'd487740c-5463-186b-9af0-466b341a7135', 'tenantName': 'testGDL'}, {'tenantId': '6f4e381a-1752-4b84-8a59-6ed6391614cf', 'tenantName': 'testTenant'}], 'statusCode': 790200, 'statusDescription': 'Success.'}
{'domains': [{'domainId': '850ff5e9-c639-404d-85a3-d920dbee509c', 'domainName': 'Support and Service'}, {'domainId': '0201adc4-ae96-46f0-ae3d-01cdba9e41d6', 'domainName': 'GE Test'}], 'statusCode': 790200, 'statusDescription': 'Success.'}
fb24f3f0-81a7-1929-4b8f-99106c23fa5b 0201adc4-ae96-46f0-ae3d-01cdba9e41d6


In [4]:
API_Body = {
    "domain_setting": {
        "tenant_id": "fb24f3f0-81a7-1929-4b8f-99106c23fa5b",
        "domain_id": "0201adc4-ae96-46f0-ae3d-01cdba9e41d6"
    },
    "basic_setting": {
        "triggered_by": "Netbrain",
        "user_id": "admin",
        "user": "gdluserTest",
        "device": "R20",
        "interface": "Ethernet2/0.100",
        "stub_name": "stubTest1",
        "stub_setting": {
            "mode": 0,
            "max_waiting_hours": 1
        }
    },
    '''"map_setting": {
        "map_create_mode": 0,
        "map_open_para": {
            "map_id": "",
            "site_id": "",
            "device_group_id": "",
            "duplicate_map": "",
            "path": ""
        },
        "map_device_para": {
            "device": "",
            "include_neighbor": false,
            "interfaces": [""],
            "neighbor_type": ""
        },
        "map_device_sitemap_para": {
            "device": "",
            "duplicate_map": false
        },
        "map_path_para": {
            "source": "",
            "source_gateway": "",
            "source_gateway_dev": "",
            "source_gateway_intf": "",
            "source_port": "",
            "destination": "",
            "destination_gateway": "",
            "destination_port": 220,
            "destination_gateway_dev": "",
            "destination_gateway_intf": "",
            "direction": 1,
            "protocol": 28,
            "protocol_name": "",
            "path_analysis_set_name": "",
            "path_analysis_set": "",
            "dataSource": {
                "type": 1,
                "recent": {
                    "unit": 2,
                    "duration": 5
                },
                "range": {
                    "start": "",
                    "end": ""
                },
                "around": {
                    "time": ""
                }
            }
        }
    },
    "runbook_setting": {},'''
    "CLI_settings": [{
            "retrieveData": {
            "showCommand": ["show version", "show interface"]
        }
    }],
    "retrieve_settings": [{
        "retrieveData": {
            "macTable": True,
            "arpTable": False,
            "deviceInfo": True,
            "interfaceInfo": True,
            "config": True,
            "bgpNbr": False,
            "routeTable": True,
            "stpTable": False,
            "cdpTable": False,
            "nctTable": ["FabricPath Route Table", "MPLS LFIB"],
            "showCommand": ["show version", "show interface"]
        }
    }],
    "ping_settings": [{
        "source_type": 2,
        "source": "R20",
        "source_interface": "Ethernet2/0.100",
        "destination": "123.10.1.6",
        "destination_interface": "Ethernet0/1",
        "timeout_seconds": 50,
        "packet_bytes": 50,
        "packet_count": 50,
        "fragment": False
    }],
    "tracert_settings": [{
        "source_type": 2,
        "source": "R20",
        "source_interface": "Ethernet2/0.100",
        "destination": "123.10.1.6",
        "destination_interface": "Ethernet0/1",
        "max_hops": 50
    }]
}

# Trigger API function
def TriggerTask(API_Body):
 
    # Trigger  API url
    API_URL = r"/ServicesAPI/API/V1/Triggers/Run"
    # Trigger API payload
 
    api_full_url = host_url + API_URL
    api_result = requests.post(api_full_url, data=json.dumps(
        API_Body), headers=headers, verify=False)
    if api_result.status_code == 200:
        return api_result.json()
    else:
        return api_result.json()
    
result = TriggerTask(API_Body)
result

{'mapId': 'fc100632-cc31-46a2-b0e9-7a9bbadc4c20',
 'mapName': 'stubTest1-20190211132830',
 'mapType': 1,
 'mapUrl': 'http://192.168.28.79/map.html?t=fb24f3f0-81a7-1929-4b8f-99106c23fa5b&d=0201adc4-ae96-46f0-ae3d-01cdba9e41d6&id=fc100632-cc31-46a2-b0e9-7a9bbadc4c20&maptype=1'}